In [76]:
from fenics import * 
import matplotlib.pyplot as plt
import numpy as np
import time

#suppress some output
import logging
logging.getLogger('FFC').setLevel(logging.WARNING)

In [77]:
## Simulation and mesh parameters

# mesh size
Lx = 1.0         
Ly = 1.0

# set grid size
mx = 20
my = int(mx*Ly)

# delta slice
J = 11
DELTA = np.linspace(0.0,1.0,J);

# number of eigenvalues
evals = 2*2*int(Lx*Ly)
E = np.zeros((2,J,evals))

In [78]:
## Mesh

# set up honeycomb lattice
q = 4*np.pi/(sqrt(3))
k1 = q*np.array([1,sqrt(3)])/2.0
k2 = q*np.array([1,-sqrt(3)])/2.0
k3 = -(k1+k2)
K = (k1-k2)/3

v1_h = np.array([sqrt(3),1])/2.0
v2_h = np.array([sqrt(3),-1])/2.0

# different edges
# a1 = 1; b1 = 1; a2 = 0; b2 = 1;     # armchair
a1, b1, a2, b2 = 1, 0, 0, 1       # zigzag
# a1 = 3; b1 = 2; a2 = 1; b2 = 1;     # general
# a1 = 2; b1 = 1; a2 = 1; b2 = 1;     # general

v1 = a1*v1_h + b1*v2_h
v2 = a2*v1_h + b2*v2_h

ktilde = -b1*k1+a1*k2         # general

# quasimomentum slice
quasimomentum = np.linspace(0.0,0.5,2)
boundary_0 = K[0]*np.ones((1,2)) + ktilde[0]*quasimomentum
boundary_1 = K[1]*np.ones((1,2)) + ktilde[1]*quasimomentum
boundary = np.concatenate((boundary_0,boundary_1),axis=0)

# create mesh      
def honeycomb_cylinder(x, y):
    return [v1[0]*x + v2[0]*y, v1[1]*x + v2[1]*y]

mesh = RectangleMesh(Point(-Lx/2,-Ly/2),Point(Lx/2,Ly/2),mx,my)#, 'left') # “left”, “right”, “left/right”, “crossed”
x = mesh.coordinates()[:,0]
y = mesh.coordinates()[:,1]

x_hat, y_hat = honeycomb_cylinder(x, y)
xy_hat_coor = np.array([x_hat, y_hat]).transpose()
mesh.coordinates()[:] = xy_hat_coor
#plot(mesh), interactive()

In [79]:
## Boundary Conditions

# sub domain for Dirichlet boundary condition
class DirichletBoundary(SubDomain):
    def inside(self, x, on_boundary):
        detv = v1[0]*v2[1] - v2[0]*v1[1]
        y_r = (-v1[1]*x[0] + v1[0]*x[1])/detv
        return bool( ( near(y_r, -Ly/2.0) or near(y_r, Ly/2.0) ) and on_boundary)
    
# sub domain for Periodic boundary condition
class PeriodicBoundary(SubDomain):

    # left boundary is "target domain" G
    def inside(self, x, on_boundary):
        detv = v1[0]*v2[1] - v2[0]*v1[1]
        x_r = (v2[1]*x[0] - v2[0]*x[1])/detv
        return bool( ( near(x_r, -Lx/2.0) and near(x_r, -Lx/2.0) ) and on_boundary)

    # map right boundary (H) to left boundary (G)
    def map(self, x, y):
        detv = v1[0]*v2[1] - v2[0]*v1[1]
        x_r = (v2[1]*x[0] - v2[0]*x[1])/detv
        y_r = (-v1[1]*x[0] + v1[0]*x[1])/detv 
        
        x_h = x_r - (Lx - 0)
        y_h = y_r
        
        y[0] = v1[0]*x_h + v2[0]*y_h
        y[1] = v1[1]*x_h + v2[1]*y_h
        
## perfectly periodic problem

# sub domain for Periodic boundary condition
class PerfectlyPeriodicBoundary(SubDomain):

    # left boundary is "target domain" G
    def inside(self, x, on_boundary):
        detv = v1[0]*v2[1] - v2[0]*v1[1]
        x_r = (v2[1]*x[0] - v2[0]*x[1])/detv
        y_r = (-v1[1]*x[0] + v1[0]*x[1])/detv 
        
        return bool( (near(x_r, -Lx/2.0) or near(y_r, -Ly/2.0)) and 
                    (not ((near(x_r, -Lx/2.0) and near(y_r, +Ly/2.0)) or 
                            (near(x_r, +Lx/2.0) and near(y_r, -Ly/2.0))) ) and on_boundary)
        
    # map right boundary (H) to left boundary (G)
    def map(self, x, y):
        detv = v1[0]*v2[1] - v2[0]*v1[1]
        x_r = (v2[1]*x[0] - v2[0]*x[1])/detv
        y_r = (-v1[1]*x[0] + v1[0]*x[1])/detv 
        
        if near(x_r, +Lx/2.0) and near(y_r, +Ly/2.0):
            x_h = x_r  - Lx - 0.0
            y_h = y_r - Ly - 0.0
        elif near(x_r, +Lx/2.0):
            x_h = x_r  - Lx - 0.0
            y_h = y_r
        else:   # near(y_r, +Ly/2.0)
            x_h = x_r 
            y_h = y_r - Ly - 0.0
            
        y[0] = v1[0]*x_h + v2[0]*y_h
        y[1] = v1[1]*x_h + v2[1]*y_h

In [80]:
## Finite Elements

# periodic = PeriodicBoundary()
periodic = PerfectlyPeriodicBoundary()

#V = FunctionSpace(mesh, "CG", 2, constrained_domain=periodic)

P2 = FiniteElement("P", triangle, 2)
element = MixedElement([P2, P2])
W = FunctionSpace(mesh, element, constrained_domain=periodic)

# define functions
(u_r, u_i) = TrialFunctions(W)
(v_r, v_i) = TestFunctions(W)

In [81]:
## Potential terms
V_e = Expression('4.0-1.0*(cos(k10*x[0]+k11*x[1]) + cos(k20*x[0]+k21*x[1]) + cos(k30*x[0]+k31*x[1]))', \
                 k10=k1[0], k11=k1[1], k20=k2[0], k21=k2[1], k30=k3[0], k31=k3[1], degree=1)
W_o = Expression('sin(k10*x[0]+k11*x[1]) + sin(k20*x[0]+k21*x[1]) + sin(k30*x[0]+k31*x[1])', \
                 k10=k1[0], k11=k1[1], k20=k2[0], k21=k2[1], k30=k3[0], k31=k3[1], degree=1)
V_e_2 = Expression('cos(k10*x[0]+k11*x[1]) + cos(k20*x[0]+k21*x[1]) + cos(k30*x[0]+k31*x[1])', \
                 k10=k1[0], k11=k1[1], k20=k2[0], k21=k2[1], k30=k3[0], k31=k3[1], degree=1)

# quasimomentum
k = Constant(K)
#k = Constant((0,0))
#k = Constant(K+(1/2.0-1/3.0)*k1)

In [82]:
## Computations

# assemble mass matrix
m_r = (u_r*v_r + u_i*v_i)*dx
m_i = (-u_r*v_i + u_i*v_r)*dx
M = PETScMatrix()
assemble(m_r+m_i, tensor=M)

#L = inner(Constant(1.0), v_r+v_i)*dx
#b = PETScVector()
#assemble_system(m_r+m_i, L, A_tensor=M, b_tensor=b)

# eigenvalue solver
def compute_evals(A, E, M, l, r):
    
    # create eigensolver
    eigensolver = SLEPcEigenSolver(A,M)
    eigensolver.parameters['spectrum'] = 'smallest real'
    eigensolver.parameters['solver']   = 'krylov-schur'
    eigensolver.parameters['tolerance'] = 1.e-10   # 1.e-12
    #eigensolver.parameters['maximum_iterations'] = 1000
    #eigensolver.parameters['problem_type'] = 'gen_hermitian'


    # solve for eigenvalues
    eigensolver.solve(evals)
    
    for i in range(0,evals):
        if i <= (eigensolver.get_number_converged()-1):
            E[l][r][i] = eigensolver.get_eigenvalue(i)[0]
    
    #return np.sort(E)
    return E

program_starts = time.time()

r = 0

# main computation
for delta in DELTA:
    
    l = 0
    #time reversal symmetry breaking
    C_r = as_matrix( ((V_e, 0.0), (0.0, V_e)) )
    C_i = as_matrix( ((0.0, -delta*V_e_2), (delta*V_e_2, 0.0)) )
    
    for k_vec in boundary.transpose():
    
        k = Constant(k_vec)

        # define problem 
        a_r = ( inner(C_r*(grad(u_r)-k*u_i), grad(v_r)-k*v_i) + inner(C_r*(grad(u_i)+k*u_r), grad(v_i)+k*v_r) +\
               -inner(C_i*(grad(u_i)+k*u_r), grad(v_r)-k*v_i) + inner(C_i*(grad(u_r)-k*u_i), grad(v_i)+k*v_r) )*dx
        a_i = ( inner(C_r*(grad(u_i)+k*u_r), grad(v_r)-k*v_i) - inner(C_r*(grad(u_r)-k*u_i), grad(v_i)+k*v_r) +\
               +inner(C_i*(grad(u_r)-k*u_i), grad(v_r)-k*v_i) + inner(C_i*(grad(u_i)+k*u_r), grad(v_i)+k*v_r) )*dx

        # assemble stiffness matrix
        A = PETScMatrix()
        assemble(a_r+a_i, tensor=A)

        # assemble_system(a_r+a_i, L, bcs, A_tensor=A, b_tensor=b)

        E = compute_evals(A, E, M, l, r)
        
        l += 1

    r += 1
    print r/float(J)
    
print time.time() - program_starts

0.0909090909091
0.181818181818
0.272727272727
0.363636363636
0.454545454545
0.545454545455
0.636363636364
0.727272727273
0.818181818182
0.909090909091
1.0
86.4375691414


In [83]:
for x in range(0,3,2):
    Emin = [1e4]*J
    Emax = [0]*J
    for Eslice in E:
        for j in range(len(Eslice)):
            #newMin = min([e for e in Eslice[j] if e>0])
            if Eslice[j][x] == 0.0:
                continue
            if Eslice[j][x] < Emin[j]:
                Emin[j] = Eslice[j][x]
            if Eslice[j][x] > Emax[j]:
                Emax[j] = Eslice[j][x]
    print 'ymin =', Emin, ';'
    print ''
    print 'ymax =', Emax, ';'
    print ''
  
# MATLAB plotting
# x = [ymin(1),ymax,ymin]
# y = [0,linspace(0,1,11),linspace(1,0,11)]
# figure; fill(x,y,'k')
# hold on; fill(x,y,'k')

ymin = [16.530938706935174, 16.514118045505537, 16.463485621300695, 16.378526430980312, 16.258369774958791, 16.10177039731478, 15.90708148111581, 15.672218887715575, 15.394615968233508, 15.071168190774634, 14.698166914139179] ;

ymax = [62.417415779303155, 60.782357781724635, 58.992159103824335, 57.106326276676491, 55.127803370871447, 53.0590278319278, 50.902144090418822, 48.659064148120493, 46.331505318114779, 43.921020123523135, 41.429021699103231] ;

ymin = [73.145577842684474, 73.143723045302949, 65.551966222359155, 66.926734448277699, 68.186482479278695, 69.325396543001503, 70.336836614667789, 71.213074168663312, 71.944650123182925, 72.516838713079451, 72.856706390886217] ;

ymax = [109.14698489661025, 108.74190388699783, 107.64560238098923, 106.07177195236606, 104.1760330022765, 102.04705478896032, 99.734913507015534, 97.270196989410437, 94.673606734302737, 91.960618001315098, 89.143808249787341] ;



In [84]:
E

array([[[  62.41741578,   62.53416999,   73.14557784,    0.        ],
        [  60.78235778,   64.06747815,   73.14372305,    0.        ],
        [  58.9921591 ,   58.9921591 ,   65.55196622,   73.13815033],
        [  57.10632628,   57.10632628,   66.92673445,   73.12883603],
        [  55.12780337,   55.12780337,   68.18648248,   73.11574625],
        [  53.05902783,   53.05902783,   69.32539654,   73.09885057],
        [  50.90214409,   50.90214409,   70.33683661,   73.07816048],
        [  48.65906415,   48.65906415,   71.21307417,   73.05385276],
        [  46.33150532,   46.33150532,   71.94465012,   73.02677828],
        [  43.92102012,   43.92102012,   72.51683871,   73.00185688],
        [  41.4290217 ,   41.4290217 ,   72.85670639,    0.        ]],

       [[  16.53093871,   16.53093871,  109.1469849 ,    0.        ],
        [  16.51411805,   16.51411805,  108.74190389,    0.        ],
        [  16.46348562,   16.46348562,  107.64560238,    0.        ],
        [  16.3785